# Implementing a port to scipy optimizers

The following notebook will guide you through the process of implementing a port to `scipy` optimizers in the f3dasm framework. We will cover how to use `scipy.optimize.minimize` to optimize a given function, and how to integrate this with the f3dasm data generation and optimization framework. This notebook intents to provide a clear and concise example of how to implement a custom optimization block.

In [1]:
from f3dasm import Block, ExperimentData
from f3dasm.design import Domain
from f3dasm.datageneration import DataGenerator
from typing import Optional, Callable
import numpy as np

The `scipy.optimize.minimize` contains a wide range of optimization algorithms, which can be used to minimize a given function. In order to use these optimizers within the `f3dasm` framework, we need to implement a custom block that wraps the `scipy.optimize.minimize` function. This block will handle the optimization process and integrate it with the data generation and optimization framework of `f3dasm`.

In [2]:
import scipy.optimize

## Writing the callback function

The `scipy.optimize.minimize` function allows us to pass a callback function that will be called at each iteration of the optimization process. This callback function can be used to store the intermediate results of the optimization, which can then be used for analysis or visualization.

```python
def callback(intermediate_result: scipy.optimize.OptimizeResult
                ) -> None:
    history_x.append(
        {input_name: intermediate_result.x for input_name
            in data.domain.input_names})
    history_y.append(
        {self.output_name: intermediate_result.fun})
```

We can initialize the two lists, `history_x` and `history_y`, to store the input and output values at each iteration. The `intermediate_result` object contains the current state of the optimization, including the current input values (`x`) and the current output value (`fun`).
As you can see, the names of the parameters are taken from the `data.domain.input_names` and the output name is taken from `self.output_name`. This allows us to store the results in a structured way, which can be easily accessed later.

## Implementing the `arm` method

The `arm` method of the `Block` will take arguments the `DataGenerator` and the `ExperimentData`, for which the latter will be used to extract the last `ExperimentSample` as the initial point of the optimization. The `arm` method will also take an optional `grad_f` argument, which can be used to provide the gradient of the function to be optimized.

```python
def arm(self, data: ExperimentData, data_generator: DataGenerator,
        output_name: str):

    self.data_generator = data_generator

    self.output_name = output_name
    input_name = data.domain.input_names[0]
    experiment_sample = data.get_experiment_sample(data.index[-1])
    self._x0 = experiment_sample.input_data[input_name]
```

## Implementing the `call` method

The `call` method of the `Block` will optimize with the given method and the initial point, and will return the `ExperimentData` with the results of the optimization. The `scipy.optimize.minimize` function will be called with the `fun` argument set to the function to be optimized, and the `x0` argument set to the initial point. The `callback` function will be passed as an argument to store the intermediate results.

```python
def call(self, data: ExperimentData, n_iterations: Optional[int] = None,
            grad_f: Optional[Callable] = None, **kwargs) -> ExperimentData:
    history_x, history_y = [], []

    def callback(intermediate_result: scipy.optimize.OptimizeResult,
                    input_name: str, output_name: str
                    ) -> None:
        history_x.append(
            {input_name: intermediate_result.x for input_name
                in data.domain.input_names})
        history_y.append(
            {self.output_name: intermediate_result.fun})

    _ = scipy.optimize.minimize(
        fun=self.data_generator.f,
        x0=self._x0,
        method=self.method,
        jac=grad_f,
        bounds=self.bounds,
        options={**self.hyperparameters},
        callback=callback,
    )

    return ExperimentData(
        domain=data.domain,
        input_data=history_x,
        output_data=history_y,
        project_dir=data.project_dir)
```

## Putting it all together

The complete implementation looks like this:

In [3]:
class ScipyOptimizer(Block):
    def __init__(
        self,
        method: str,
        bounds: Optional[scipy.optimize.Bounds] = None,
        **hyperparameters,
    ):
        self.bounds = bounds
        self.method = method
        self.hyperparameters = hyperparameters

    def arm(
        self,
        data: ExperimentData,
        data_generator: DataGenerator,
        output_name: str,
    ):
        self.data_generator = data_generator

        self.output_name = output_name
        input_name = data.domain.input_names[0]
        experiment_sample = data.get_experiment_sample(data.index[-1])
        self._x0 = experiment_sample.input_data[input_name]

    def call(
        self,
        data: ExperimentData,
        n_iterations: Optional[int] = None,
        grad_f: Optional[Callable] = None,
        **kwargs,
    ) -> ExperimentData:
        history_x, history_y = [], []

        def callback(
            intermediate_result: scipy.optimize.OptimizeResult,
        ) -> None:
            history_x.append(
                {
                    input_name: intermediate_result.x
                    for input_name in data.domain.input_names
                }
            )
            history_y.append({self.output_name: intermediate_result.fun})

        _ = scipy.optimize.minimize(
            fun=self.data_generator.f,
            x0=self._x0,
            method=self.method,
            jac=grad_f,
            bounds=self.bounds,
            options={**self.hyperparameters},
            callback=callback,
        )

        return ExperimentData(
            domain=data.domain,
            input_data=history_x,
            output_data=history_y,
            project_dir=data.project_dir,
        )

## Example implementation

In [4]:
from f3dasm import datagenerator
from f3dasm import create_sampler


# Create a simple data generator function
# that computes y = 0.5 * sum(x^4 - 16 * x^2 + 5 * x)
# This will be used to evaluate the samples generated by the optimizer.
@datagenerator(output_names="y")
def f(x: np.ndarray):
    y = 0.5 * np.sum(x**4 - 16 * x**2 + 5 * x)
    return y


# Create your custom OptunaOptimizer and give it the optuna sampler.
optimizer = ScipyOptimizer(method="cg")

# Initialize the domain and experiment data
domain = Domain()
domain.add_array(name="x", shape=(10,), low=-10, high=10)
domain.add_output("y")
data = ExperimentData(domain=domain)

# Sample random initial points
sampler = create_sampler("random")
data = sampler.call(data, n_samples=1)

# Evaluate the initial points using the data generator
f.arm(data)
data = f.call(data)

# Arm the optimizer with the experiment data and data generator
optimizer.arm(data, data_generator=f, output_name="y")

# Run the optimization for a specified number of iterations
data += optimizer.call(data, n_iterations=10)

In [5]:
data

,jobs,input,output
,,x,y
0,finished,"[7.980457519091495, 2.4031542981285465, 1.1027...",1626.173731
1,finished,"[3.0539356753027205, 2.4484402060218255, 1.171...",-103.470488
2,finished,"[2.2286113715705413, 3.0181781344055145, 2.185...",-116.203495
3,finished,"[3.020890411448555, 2.3569857659522095, 3.0188...",-161.026406
4,finished,"[3.2900131787972873, 2.1904036979559067, 3.327...",-214.933392
5,finished,"[3.15844415645565, 2.2551167752490495, 3.19002...",-239.520289
6,finished,"[2.620057629601515, 2.6213025560903325, 2.6049...",-294.629662
7,finished,"[2.6497636859942393, 2.775869703001976, 2.6395...",-304.928880
8,finished,"[2.7330042278249453, 2.74908002390725, 2.73101...",-306.814436
